# P300 Speller Dataset - BCI Competition III

In [ ]:
import numpy as np
from scipy.io import loadmat
import os

# Directory containing the .mat files
data_dir = 'BCI_Comp_III_Wads_2004'

def load_p300_mat(mat_path):
    """
    Load a P300 Speller .mat file and return the relevant NumPy arrays.
    This keeps the natural structure: 
    - Signal: (epochs, samples, channels)
    - Flashing / StimulusCode / StimulusType: (epochs, samples)
    """

    data = loadmat(mat_path)

    # Extract variables and convert to float64 / int
    Signal       = data['Signal'].astype(np.float64)          # EEG data
    Flashing     = data['Flashing'].astype(np.int16)          # 0/1 flash indicator
    StimulusCode = data['StimulusCode'].astype(np.int16)      # 1–12 code
    StimulusType = data['StimulusType'].astype(np.int16) if 'StimulusType' in data else None
    TargetChar   = data['TargetChar'] if 'TargetChar' in data else None

    return {
        "Signal": Signal,                   # (epochs, samples, channels)
        "Flashing": Flashing,               # (epochs, samples)
        "StimulusCode": StimulusCode,       # (epochs, samples)
        "StimulusType": StimulusType,       # (epochs, samples) or None
        "TargetChar": TargetChar            # (epochs,) or None
    }


# Load all 4 .mat files
mat_files = {
    'A_train': os.path.join(data_dir, 'Subject_A_Train.mat'),
    'A_test':  os.path.join(data_dir, 'Subject_A_Test.mat'),
    'B_train': os.path.join(data_dir, 'Subject_B_Train.mat'),
    'B_test':  os.path.join(data_dir, 'Subject_B_Test.mat')
}

all_data = {}

print("Loading P300 datasets...\n")
for key, path in mat_files.items():
    print(f"Loading {key} from {path}")
    all_data[key] = load_p300_mat(path)

Loading P300 datasets...

Loading A_train from BCI_Comp_III_Wads_2004/Subject_A_Train.mat
Loading A_test from BCI_Comp_III_Wads_2004/Subject_A_Test.mat
Loading B_train from BCI_Comp_III_Wads_2004/Subject_B_Train.mat
Loading B_test from BCI_Comp_III_Wads_2004/Subject_B_Test.mat

Done! Your datasets are now stored as NumPy arrays.
Example: all_data['A_train']['Signal'].shape


In [ ]:
all_data['A_train']['Signal'].shape



(85, 7794)